In [ ]:
import cv2
import queue
import numpy as np
import matplotlib.pyplot as plt
from iplib import *

In [ ]:
img = imgRead("1.png")
img = cv2.resize(img, np.array(img.shape[:2])[::-1]//5)
imgPlot(img)
plt.show()

In [ ]:
def bfs(mask: np.ndarray, start: list, img: np.ndarray) -> np.ndarray:
  res = img.copy()
  q = queue.Queue()
  q.put(start)
  while not q.empty():
    y, x = q.get()
    if mask[y,x] == 0 and res[y,x] != 255:
      res[y, x] = 255
      if x - 1 >= 0:
        q.put([y, x - 1])
      if x + 1 < w:
        q.put([y, x + 1])
      if (y - 1 >= 0):
        q.put([y - 1, x])
      if (y + 1 < h):
        q.put([y + 1, x])
  return res

def houghTrans(rho, theta):
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = rho * a
    y0 = rho * b
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * a)
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * a)
    return [(x1,y1), (x2,y2)]

In [ ]:
gray = bgr2gray(img)
edge = cv2.Canny(gray, 250, 255)
bina = binary(gray, 170)
figSize(30,20)
imgs = [img, gray, bina, edge]
imgPlotTable(imgs, len(imgs), 1)

In [ ]:
lines = cv2.HoughLines(edge, 1, np.pi / 180, 120)
left_bounds = []
right_bounds = []
for line in lines:
    rho, theta = line[0]
    rad = np.rad2deg(theta)
    if rad > 30 and rad < 70:
        left_bounds.append([rho, theta])
    elif rad > 90:
        right_bounds.append([rho, theta])
    else:
        continue

left_bound = min(left_bounds, key = lambda x: x[1])
right_bound = max(right_bounds, key = lambda x: x[1])

mask1 = np.zeros_like(edge)
tmp = img.copy()
for rho, theta in left_bounds + [right_bound]:
    p1, p2 = houghTrans(rho, theta)
    cv2.line(mask1, p1, p2, (255), 1)
    cv2.line(tmp, p1, p2, (0, 0, 255), 1)
h, w = mask1.shape
start_point = [h-1, w//2]
res1 = bfs(mask1, start_point, gray)

mask2 = np.zeros_like(edge)
for rho, theta in left_bounds:
    p1, p2 = houghTrans(rho, theta)
    cv2.line(mask2, p1, p2, 255, 1)

xmean = np.mean(np.arange(w)[mask2[h-1,:] > 0], dtype=np.intp)
res2 = bfs(mask2, [h-1, xmean], gray)

res2[res2 == 255] = 0
lst = [tmp, res1, res2]
figSize(20,20)
imgPlotTable(lst, len(lst), 1)